In [125]:
# %%writefile Factor_Cacher.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import warnings
import pandas as pd
import numpy as np

import tools.Sample_Tools as smpl
import base.JuUnits as ju


__cache_dir__ = ju.get_root_path()+'/cache.feather/'
__cache_type__ = '.feather'

# df = smpl.get_sample_by_zs(name='沪深300', end='2021-11-28', gap=2500,  only_main=True, filter_st=True).data
# df.to_parquet(__cache_dir__+'test.parquet')

def file_path(file_name):
    return __cache_dir__ + file_name + __cache_type__

def is_cache_exist(cache_name):
    return os.path.exists(file_path(cache_name))

def save_cache(name, data):
    df.reset_index().to_feather(file_path(cache_name))
    
def load_cache(cache_name:str, to_series:bool=False):
    assert is_cache_exist(cache_name),'cache not exist, create first'
    df = pd.read_feather(file_path(cache_name)).set_index(['date','code'])
    if to_series:
        df =  df.squeeze()
    return df

def load_cache_adv(cache_name:str, start:str, end:str, to_series:bool=False, flag_time:str='date'):
    assert is_cache_exist(cache_name),'cache not exist, create first'
        
    st = pd.Timestamp(start) 
    en = pd.Timestamp(end) 

    df = pd.read_feather(file_path(cache_name)).set_index([flag_time,'code'])[st:en]
    date_idx = df.index.get_level_values(0).unique()
    if not (st >= date_idx.min() and en <= date_idx.max()):
        info = '"{}"：date range out of cache[{},{}]'.format(
            cache_name,
            date_idx.min().strftime('%Y-%m-%d %H:%M:%S'), 
            date_idx.max().strftime('%Y-%m-%d %H:%M:%S')
        )
        warnings.warn(info)
        
    df = df[st:en]
    
    if to_series:
        df =  df.squeeze()
    return df

def load_caches(cache_names:list):
    tmp = []
    for name in cache_names:
        df = load_cache(name)
        tmp.append(df)
    return pd.concat(tmp,axis=1)

In [31]:
%%time
df.reset_index().to_feather(__cache_dir__+'test.feather')

Wall time: 71.8 ms


In [126]:
# df['open'].reset_index().to_feather(__cache_dir__+'test_s.feather')
# pd.read_feather(__cache_dir__+'test_s.feather').set_index(['date','code']).squeeze()
# df.squeeze()
# load_cache_adv('test','2011-11-11','2022-11-11')
load_caches(['test','test-Copy1'])

open    high     low   close     volume        amount  \
date       code                                                              
2011-08-15 000001   16.65   17.25   16.37   17.13   308527.0  5.191207e+08   
           000002    8.43    8.55    8.33    8.50   444015.0  3.747184e+08   
           000063   18.15   18.65   18.09   18.34   291095.0  5.340780e+08   
           000066    6.36    6.52    6.31    6.44    75867.0  4.869003e+07   
           000069    7.85    7.98    7.77    7.90   128491.0  1.012825e+08   
...                   ...     ...     ...     ...        ...           ...   
2021-11-26 603899   58.76   59.10   58.05   58.30    27712.0  1.618647e+08   
           603939   47.39   47.77   46.82   47.28    29457.0  1.393297e+08   
           603986  156.34  157.00  150.79  151.99   143993.0  2.198587e+09   
           603993    5.85    6.00    5.81    5.89  2392058.0  1.415083e+09   
           605499  188.29  189.98  184.56  187.00    10644.0  1.987549e+08   

                     open    high     low   close     volume        amount  
date       code                                                             
2011-08-15 000001   16.65   17.25   16.37   17.13   308527.0  5.191207e+08  
           000002    8.43    8.55    8.33    8.50   444015.0  3.747184e+08  
           000063   18.15   18.65   18.09   18.34   291095.0  5.340780e+08  
           000066    6.36    6.52    6.31    6.44    75867.0  4.869003e+07  
           000069    7.85    7.98    7.77    7.90   128491.0  1.012825e+08  
...                   ...     ...     ...     ...        ...           ...  
2021-11-26 603899   58.76   59.10   58.05   58.30    27712.0  1.618647e+08  
           603939   47.39   47.77   46.82   47.28    29457.0  1.393297e+08  
           603986  156.34  157.00  150.79  151.99   143993.0  2.198587e+09  
           603993    5.85    6.00    5.81    5.89  2392058.0  1.415083e+09  
           605499  188.29  189.98  184.56  187.00    10644.0  1.987549e+08  

[524045 rows x 12 columns]